In [51]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import importlib
importlib.reload(prep)
importlib.reload(pw)

<module 'features_pairwise' from '/home/jacob/JUPYTER PROJECTS/ML/project/features_pairwise.py'>

In [55]:
import features_nodewise as nw
import features_pairwise as pw
import preprocessing as prep

import importlib
import matplotlib.pyplot as plt
import random
import numpy as np
import igraph
from sklearn import preprocessing
import nltk
import csv
from functools import reduce
from sklearn import metrics
from nltk.tokenize import RegexpTokenizer
import pickle

## Read train / test node pairs
nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("./data/train_train_set.csv", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

with open("./data/train_test_set.csv", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)
    
with open("./data/testing_set.csv", "r") as f:
    reader = csv.reader(f)
    competition_set  = list(reader)
    
competition_set = [element[0].split(" ") for element in competition_set]

random.seed(0)
training_set = [element[0].split(" ") for element in training_set]
to_keep_train = random.sample( range(len(training_set)),k=int(round(len(training_set)*0.05)) )
training_set_reduced = [training_set[i] for i in to_keep_train]

testing_set = [element[0].split(" ") for element in testing_set]
to_keep_test = random.sample( range(len(testing_set)),k=int(round(len(testing_set)*1)) )
testing_set_reduced = [testing_set[i] for i in to_keep_test]

with open("./data/node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

corpus = [element[5] for element in node_info]
titles = [element[2] for element in node_info]

t_titles = prep.tfidf(titles)
t = prep.tfidf(corpus)
l = nw.LSA(t,n_components=100)

t_ngrams = prep.tfidf(corpus, r= (2,3), midf = 2, madf=0.5,feats=150000, sublinear = True)
l_ngrams = nw.LSA(t_ngrams,n_components=300)

IDs = [element[0] for element in node_info]
node_dict = prep.to_dict( [element[0] for element in node_info],range(len(node_info)) )
index_dict = prep.to_dict(range(len(IDs)),IDs)

[nltk_data] Downloading package punkt to /home/jacob/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jacob/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
n_samples: 27770, n_features: 2890
n_samples: 27770, n_features: 10000
Performing dimensionality reduction using LSA
Explained variance of the SVD step: 16%
n_samples: 27770, n_features: 150000
Performing dimensionality reduction using LSA
Explained variance of the SVD step: 7%


In [6]:
title_sim = t_titles*t_titles.transpose()

In [5]:
import copy
# create graphs (train vs test), (reduced vs full)

train_IDs = set([training_set[i][0] for i in range(len(training_set))])
train_IDs = train_IDs | set([training_set[i][1] for i in range(len(training_set))])
train_IDs = list(train_IDs) #igraph doesn't like sets...
train_edges = [(element[0],element[1]) for element in training_set]
# train_graph = prep.article_graph(train_IDs,train_edges)

train_IDs_reduced = set([training_set_reduced[i][0] for i in range(len(training_set_reduced))])
train_IDs_reduced = train_IDs_reduced | set([training_set_reduced[i][1] for i in range(len(training_set_reduced))])
train_IDs_reduced = list(train_IDs_reduced)
train_edges_reduced = [(element[0],element[1]) for element in training_set_reduced]
train_graph_reduced = prep.article_graph(train_IDs_reduced,train_edges_reduced)

gold_train_graph_reduced = copy.copy(train_graph_reduced)
to_del = [(element[0],element[1]) for element in training_set_reduced if element[2]=='0']
gold_train_graph_reduced.delete_edges(to_del)

test_IDs = set([testing_set[i][0] for i in range(len(testing_set))])
test_IDs = test_IDs | set([testing_set[i][1] for i in range(len(testing_set))])
test_IDs = list(test_IDs)
test_edges = [(element[0],element[1]) for element in testing_set]
# test_graph = prep.article_graph(test_IDs,test_edges)

# all_IDs = set(test_IDs) | set(train_IDs)
# all_IDs = list(all_IDs)
# all_edges = train_edges
# all_edges.extend(test_edges)
# all_graph = prep.article_graph(all_IDs,all_edges)

# test_IDs_reduced = set([testing_set_reduced[i][0] for i in range(len(testing_set_reduced))])
# test_IDs_reduced = test_IDs_reduced | set([testing_set_reduced[i][1] for i in range(len(testing_set_reduced))])
# test_IDs_reduced = list(test_IDs_reduced)
# test_edges_reduced = [(element[0],element[1]) for element in testing_set_reduced]
# test_graph_reduced = prep.article_graph(test_IDs_reduced,test_edges_reduced)

competition_IDs = set([competition_set[i][0] for i in range(len(competition_set))])
competition_IDs = competition_IDs | set([competition_set[i][1] for i in range(len(competition_set))])
competition_IDs = set(competition_IDs) | set(test_IDs) | set(train_IDs) 
competition_IDs = list(competition_IDs)
competition_edges = [(element[0],element[1]) for element in competition_set]
competition_edges.extend(train_edges)
competition_edges.extend(test_edges)
competition_graph = prep.article_graph(competition_IDs,competition_edges)

In [6]:
import copy
competition_graph_reduced = copy.copy(competition_graph)
to_rem = []
for i,triple in enumerate(testing_set):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    source_info = node_info[index_source]
    target_info = node_info[index_target]
    temp_diff = int(source_info[1]) - int(target_info[1])
    if temp_diff<-1:
        to_rem.append((source,target))
        
for i,triple in enumerate(training_set):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    source_info = node_info[index_source]
    target_info = node_info[index_target]
    temp_diff = int(source_info[1]) - int(target_info[1])
    if temp_diff<-1:
        to_rem.append((source,target))
    
for i,triple in enumerate(competition_set):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    source_info = node_info[index_source]
    target_info = node_info[index_target]
    temp_diff = int(source_info[1]) - int(target_info[1])
    if temp_diff<-1:
        to_rem.append((source,target))

competition_graph_reduced.delete_edges(to_rem)

In [210]:
l=[]
for node in random.sample(train_IDs,10000):
    l.append(train_graph.degree(node,mode='IN'))

m=[]
for node in random.sample(test_IDs,10000):
    m.append(test_graph.degree(node,mode='IN'))
print(np.mean(l), np.mean(m))

19.9333 2.2779


In [72]:
## Read all the features that we have stored in files
import os.path

def to_feature_shape(feat):
    feat = np.array(feat)
    if len(feat.shape) == 1:#not a real array but just a long list
        feat = np.reshape(feat,(feat.shape[0],1))
    return feat

#This method should throw an error if something goes wrong
def read_feature(file_path):
    f = open(file_path, 'rb')
    feat = to_feature_shape(pickle.load(f))
    f.close()
    return feat
    
features_to_read = ["overlap_title",
                 "comm_auth",
                 "temp_diff",
                "citation_check",
                "max_sim",
                "peer_popularity",
                "edge_check",
                "LSA_distance",
                "node_degree",
                "succ_pred",
                "title_sim",
                "temporal_fit",
                   "N_LSA_distance"]

train_features_dict = dict()
train_features_reduced_dict = dict()
test_features_dict = dict()
competition_features_dict = dict()

for name in features_to_read:
    # Train
    file_path = './current_features_train/'+name
    try:
        this_feat = read_feature(file_path)
        train_features_dict[name] = this_feat
        train_features_reduced_dict[name] = this_feat[to_keep_train,:]
    except:
        pass
    # Train reduced
    file_path = './current_features_train/'+name+'_reducedgraph'
    try:
        this_feat = read_feature(file_path)
        train_features_dict[name] = this_feat
        train_features_reduced_dict[name] = this_feat[to_keep_train,:]
    except:
        pass
    # Test
    file_path = './current_features_test/'+name
    try:
        this_feat = read_feature(file_path)
        test_features_dict[name] = this_feat
    except:
        pass
    # Test reduced
    file_path = './current_features_test/'+name+'_reducedgraph'
    try:
        this_feat = read_feature(file_path)
        test_features_dict[name] = this_feat
    except:
        pass
    file_path = './features_competition/'+name
    try:
        this_feat = read_feature(file_path)
        competition_features_dict[name] = this_feat
    except:
        pass
    # Test reduced
    file_path = './features_competition/'+name+'_reducedgraph'
    try:
        this_feat = read_feature(file_path)
        competition_features_dict[name] = this_feat
    except:
        pass

In [ ]:
np.reshape(train_true_labels,(train_true_labels.shape[0],))

In [76]:
import pickle
import numpy as np

train_true_labels = read_feature('./features_train/true_labels')
train_true_labels = np.reshape(train_true_labels,(train_true_labels.shape[0],))
train_true_labels_reduced = train_true_labels[to_keep_train]
test_true_labels = read_feature('./features_test/true_labels')
test_true_labels = np.reshape(test_true_labels,(test_true_labels.shape[0],))
test_true_labels_reduced = test_true_labels[to_keep_test]

In [58]:
############################################
###   Write features to disk - Training  ###
############################################
import os
import pickle

def write_feature_to_disk(feat,file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(feat,file)

print("Current wd: ",os.getcwd())
# write_feature_to_disk(train_features_reduced_dict['title_sim'],'./features_train/title_sim')

Current wd:  /home/jacob/JUPYTER PROJECTS/ML/project


In [ ]:
###########################################
###   Write features to disk - Testing  ###
###########################################
import os
import pickle

print("Current wd: ",os.getcwd())
# write_feature_to_disk(test_features_dict['succ_pred'],'./features_test/succ_pred')

In [80]:
#Combine all features to one vector
train_features_dict.keys()
test_features_dict.keys()
for key,feat in train_features_dict.items():
    print(key,feat.shape)
train_features = np.concatenate([train_features_dict['overlap_title'],
                                         train_features_dict['comm_auth'],
                                         train_features_dict['temp_diff'],
                                         train_features_dict['citation_check'],
                                         train_features_dict['max_sim'],
                                         train_features_dict['peer_popularity'],
                                         train_features_dict['edge_check'],
                                         train_features_dict['succ_pred'],
                                         train_features_dict['LSA_distance'],
                                         train_features_dict['title_sim'],
                                         train_features_dict['temporal_fit'],
                                        train_features_dict['N_LSA_distance']]                                            
                                        ,axis = 1)

train_features_reduced = np.concatenate([train_features_reduced_dict['overlap_title'],
                                         train_features_reduced_dict['comm_auth'],
                                         train_features_reduced_dict['temp_diff'],
                                         train_features_reduced_dict['citation_check'],
                                         train_features_reduced_dict['max_sim'],
                                         train_features_reduced_dict['peer_popularity'],
                                         train_features_reduced_dict['edge_check'],
                                         train_features_reduced_dict['succ_pred'],
                                         train_features_reduced_dict['LSA_distance'],
                                         train_features_reduced_dict['title_sim'],
                                         train_features_reduced_dict['temporal_fit'],
                                        train_features_reduced_dict['N_LSA_distance']]
                                        ,axis = 1)

test_features = np.concatenate(        [test_features_dict['overlap_title'],
                                        test_features_dict['comm_auth'],
                                        test_features_dict['temp_diff'],
                                        test_features_dict['citation_check'],
                                        test_features_dict['max_sim'],
                                        test_features_dict['peer_popularity'],
                                        test_features_dict['edge_check'],
                                        test_features_dict['succ_pred'],
                                        test_features_dict['LSA_distance'],
                                        test_features_dict['title_sim'],
                                        test_features_dict['temporal_fit'],
                                       test_features_dict['N_LSA_distance']]
                                        ,axis = 1)


competition_features = np.concatenate(  [competition_features_dict['overlap_title'],
                                        competition_features_dict['comm_auth'],
                                        competition_features_dict['temp_diff'],
                                        competition_features_dict['citation_check'],
                                        competition_features_dict['max_sim'],
                                        competition_features_dict['peer_popularity'],
                                        competition_features_dict['edge_check'],
                                        competition_features_dict['succ_pred'],
                                        competition_features_dict['LSA_distance'],
                                        competition_features_dict['title_sim'],
                                        competition_features_dict['temporal_fit'],
                                        competition_features_dict['N_LSA_distance']]
                                        ,axis = 1)

overlap_title (554602, 1)
comm_auth (554602, 1)
temp_diff (554602, 1)
citation_check (554602, 4)
max_sim (554602, 9)
peer_popularity (554602, 1)
edge_check (554602, 1)
LSA_distance (554602, 1)
node_degree (554602, 4)
succ_pred (554602, 4)
title_sim (554602, 1)
temporal_fit (554602, 2)
N_LSA_distance (554602, 1)


In [274]:
from sklearn.preprocessing import PolynomialFeatures as PF

poly = PF(degree=2, interaction_only=False)

selection = [23, 1, 2, 8, 13, 16, 20, 22]
# selection = list(range(train_features_reduced.shape[1]))

train_sp = poly.fit_transform(train_features_reduced[:,selection])
test_sp = poly.fit_transform(test_features[:,selection])


In [11]:
from sklearn.preprocessing import StandardScaler as SS
scaler = SS()
normalized_train_features_reduced = scaler.fit_transform(train_features_reduced)
normalized_test_features = scaler.fit_transform(test_features)

In [64]:
from sklearn import svm

classifier = svm.LinearSVC(loss='hinge')
# selection = [23, 1, 16, 20, 22, 6, 21]
selection = list(range(train_features_reduced.shape[1]))

# classifier.fit(train_features_reduced[:,selection], train_true_labels_reduced)
# preds_svm = list(classifier.predict(test_features[:,selection]))
classifier.fit(normalized_train_features_reduced[:,selection], train_true_labels_reduced)
preds_svm = list(classifier.predict(normalized_test_features[:,selection]))

acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_svm)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_svm)))
print('acc:',acc,'f1:',f1)

acc: 0.9613692332950254 f1: 0.9640406510277374


/home/jacob/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [63]:
from sklearn.linear_model import LogisticRegression as lr
selection = [ 1,  7,  9, 10, 11, 15, 16, 17, 19, 21, 22, 23]
model = lr(penalty='l1').fit(train_features_reduced[:,selection], train_true_labels_reduced)
preds_lg = list(model.predict(test_features[:,selection]))

# model = lr(penalty='l1').fit(train_sp, train_true_labels)
# preds_lg = list(model.predict(test_sp))

acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_lg)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_lg)))
print('acc:',acc,'f1:',f1)

/home/jacob/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


acc: 0.9622886225578723 f1: 0.9651932780749474


In [ ]:
np.argsort(list(map(lambda x: abs(x),model.coef_)))[0][:14]
# sorted(list(map(lambda x: abs(x),model.coef_))[0],reverse=True)
[poly.powers_[i,:] for i in [31,28,9,17,13]]

In [62]:
from sklearn.neighbors import KNeighborsClassifier
selection = [1, 2, 6, 13, 16, 20, 21, 22]
nNhbr = KNeighborsClassifier(n_neighbors=9,weights='distance')
nNhbr.fit(train_features_reduced[:,selection],train_true_labels_reduced) # do Ytrain.ravel() for length one Y values
preds_knn = nNhbr.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_knn)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_knn)))
print('acc:',acc,'f1:',f1)

acc: 0.9505828271219833 f1: 0.9547313962581966


In [285]:
from sklearn.tree import DecisionTreeClassifier
# selection = [0, 1, 2, 8, 13, 16, 20, 22]
selection = [ 0,  2,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 19, 20, 21, 22,23]

dTree = DecisionTreeClassifier()
dTree.fit(train_features_reduced[:,selection],train_true_labels_reduced) # do Ytrain.ravel() for length one Y values
preds_dt = dTree.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_dt)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_dt)))
print('acc:',acc,'f1:',f1)

acc: 0.9403053685765884 f1: 0.9460373998219056


In [286]:
# Joined forces
joined_DTree = DecisionTreeClassifier()

preds_test_svm = np.reshape(preds_svm,(len(preds_svm),1))
preds_test_lg = np.reshape(preds_lg,(len(preds_lg),1))
preds_test_knn = np.reshape(preds_knn,(len(preds_knn),1))
preds_test_dt = np.reshape(preds_dt,(len(preds_dt),1))
combined_preds = np.concatenate([preds_test_svm,preds_test_lg,preds_test_knn,preds_test_dt],axis=1)

joined_DTree.fit(combined_preds[0:50000,:], test_true_labels[0:50000])
preds_joined = joined_DTree.predict(combined_preds[50000:,:])

acc = metrics.accuracy_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
f1 = metrics.f1_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
print("acc:",acc,"f1:",f1)

acc: 0.9629697525206233 f1: 0.9658956609826103


In [ ]:
# Which features are important?
# Histogram of the feature frequency for all selections that reached > 90% acc
frequency = [0]*total_num_features

num_good_preds = 0
min_acc = 0.93
for i,acc in enumerate(accs):
    if acc > min_acc:
        num_good_preds += 1
        for f in feature_selections[i]:
            frequency[f] += 1
#frequency = [freq/num_good_preds for freq in frequency]
print("")
print("number of classifiers: ",len(accs))
print("number of accs >",min_acc,": ",sum([1 for acc in accs if acc > min_acc]))
plt.figure()
plt.bar(x=range(len(frequency)),height=frequency)
plt.show()

In [145]:
# Adaboost DecisionTrees
from sklearn.ensemble import AdaBoostClassifier
selection = [1, 2, 6, 13, 16, 20, 22, 23]
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=500,learning_rate=0.1)
ada.fit(train_features_reduced[:,selection],train_true_labels_reduced)
preds_ada = ada.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
print("acc:",acc,"f1:",f1)

acc: 0.956279757018552 f1: 0.9592158664522552


In [ ]:
# Adaboost LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
selection = [23, 1, 2, 8, 13, 16, 20, 22]
ada = AdaBoostClassifier(lr(),n_estimators=500,learning_rate=1)
ada.fit(train_features_reduced[:,selection],train_true_labels_reduced)
preds_ada = ada.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
print("acc:",acc,"f1:",f1)

In [61]:
# ExtraTreesClassifier
#fiddle with n_estimators and min_samples_leaf

from sklearn.ensemble import ExtraTreesClassifier
# selection = [1, 2, 6, 13, 16, 20, 22, 23]
selection = [1, 2,  5,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

#add one to min_sample_leaf for full train_features (or change to 4)
extraTrees = ExtraTreesClassifier(n_estimators=750,max_depth=90,min_samples_split=10,min_samples_leaf=0.00001)
extraTrees.fit(train_features_reduced[:,selection],train_true_labels_reduced)
preds_extra = extraTrees.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_extra)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_extra)))
print("acc:",acc,"f1:",f1)

acc: 0.9645706780495813 f1: 0.9675674050918271


In [81]:
from sklearn.ensemble import RandomForestClassifier

#f1: 0.96989, takes ~35 mins
selection = [2,  3,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

randForest = RandomForestClassifier(n_estimators = 750, min_samples_split = 5, min_samples_leaf = 4, max_depth = 60)

randForest.fit(train_features_reduced[:,selection],train_true_labels_reduced)
preds_randForest = randForest.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_randForest)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_randForest)))
print("acc:",acc,"f1:",f1)

acc: 0.9663109505828271 f1: 0.9691512071920382


In [65]:
preds_rf = list(preds_rf)

# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
preds_rf = list(zip(range(len(competition_set)), preds_rf))

with open("rf_predictions.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    for row in preds_rf:
        csv_out.writerow(row)
        
with open('random_forest_model', 'wb') as file:
        pickle.dump(randForest,file)

In [12]:
from xgboost.sklearn import XGBClassifier

selection = [0, 2, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25]
# selection = [ 2, 10, 14, 16, 19, 20, 21, 22, 23]

xgb = XGBClassifier(objective= 'binary:logistic', subsample = 0.8, colsample_bytree=0.8, learning_rate=0.01, 
                     max_depth=5, min_child_weight = 4, gamma=0, reg_lambda=2)

xgb.fit(train_features[:,selection],train_true_labels)
preds_xgb = xgb.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_xgb)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_xgb)))
print("acc:",acc,"f1:",f1)

acc: 0.9630274175012313 f1: 0.9661180152258299


In [28]:
preds_xgb = list(preds_xgb)

# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
preds_xgb = list(zip(range(len(competition_set)), preds_xgb))

with open("xgb_predictions.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    for row in preds_xgb:
        csv_out.writerow(row)

In [66]:
from sklearn.ensemble import VotingClassifier

#### NEEDS TO BE FIXED
preds_test_svm = np.reshape(preds_svm,(len(preds_svm),1))
preds_test_lg = np.reshape(preds_lg,(len(preds_lg),1))
preds_test_knn = np.reshape(preds_knn,(len(preds_knn),1))
preds_test_et = np.reshape(preds_extra,(len(preds_extra),1))
preds_test_xgb = np.reshape(preds_xgb,(len(preds_xgb),1))

combined_preds = np.concatenate([preds_test_svm,preds_test_lg,preds_test_knn,preds_test_et,preds_test_xgb],axis=1)

VotingClassifier.fit(combined_preds[0:50000,:], test_true_labels[0:50000])
preds_joined = VotingClassifier.predict(combined_preds[50000:,:])

acc = metrics.accuracy_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
f1 = metrics.f1_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
print("acc:",acc,"f1:",f1)

TypeError: fit() missing 1 required positional argument: 'y'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

#{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
# tuned_parameters = [{'penalty': ['l1'], 'C': [1, 10], 'loss': ['squared_hinge'],'dual': [False],
#                      'max_iter': [1000,5000]}]
tuned_parameters = [{'n_estimators': [500],'criterion': ['gini'],'max_depth': [None,50,100],
                    'min_samples_split': [2,10], 'min_samples_leaf': [4,0.00001], 'bootstrap': [True, False]}]
selection = [1, 2, 6, 13, 16, 20, 22, 23]

scores = ['f1_macro'] #'accuracy_score'

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(ExtraTreesClassifier(), tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(train_features_reduced[:,selection], train_true_labels_reduced)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = test_true_labels, clf.predict(test_features[:,selection])
    print(classification_report(y_true, y_pred))
    print()

In [33]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

randForest = RandomForestClassifier()
selection = [1, 2, 6, 13, 16, 20, 22, 23,25]

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 110, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,0.00001,0.000001]
# Method of selecting samples for training each tree
bootstrap = [True, False]
criterion = ['gini', 'entropy']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
                'criterion': criterion}

# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = randForest, param_distributions = random_grid, n_iter = 65,
                              cv = 2, verbose = 10, random_state=42, n_jobs = -1)

rf_random.fit(train_features_reduced[:,selection], train_true_labels_reduced)

Fitting 2 folds for each of 65 candidates, totalling 130 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 22.1min finished


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=65, n_jobs=-1,
          param_distributions={'n_estimators': [200, 650, 1100, 1550, 2000], 'max_features': ['auto', 'log2'], 'max_depth': [2, 14, 26, 38, 50, 62, 74, 86, 98, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 1e-05, 1e-06], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verb

In [50]:
from scipy.stats import mode
top = np.argsort(rf_random.cv_results_['rank_test_score'])
# top[:5]
# xgb_random.best_index_
mode([rf_random.cv_results_['params'][i]['bootstrap'] for i in top[:15]])
[rf_random.cv_results_['mean_test_score'][i] for i in top[:15]]

# [{'n_estimators': 1400,
#   'min_samples_split': 5,
#   'min_samples_leaf': 4,
#   'max_features': 'auto',
#   'max_depth': 60,
#   'criterion': 'gini',
#   'bootstrap': True},

[0.9602235845654525,
 0.9601875225387667,
 0.9601153984853948,
 0.9601153984853948,
 0.9601153984853948,
 0.960079336458709,
 0.9600432744320231,
 0.9600072124053372,
 0.9599711503786513,
 0.9599350883519654,
 0.9599350883519654,
 0.9599350883519654,
 0.9599350883519654,
 0.9599350883519654,
 0.9598990263252795]

In [56]:
from sklearn.feature_selection import RFECV

estimator = RandomForestClassifier(n_estimators = 750, min_samples_split = 5, min_samples_leaf = 4, max_depth = 60)
# = ExtraTreesClassifier(n_estimators=500,max_depth=50,min_samples_split=10,min_samples_leaf=0.00001)
# = DecisionTreeClassifier()
# = lr(penalty='l1')
# = svm.LinearSVC(loss='hinge',max_iter=10000)

random.seed(0)
short = random.sample(range(len(train_features_reduced)),10000)

selector = RFECV(estimator, step=3, cv=3,min_features_to_select=3,verbose=10)
selector = selector.fit(train_features_reduced[short], train_true_labels_reduced[short])
selector.ranking_

Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
Fitting estimator with 8 features.
Fitting estimator with 5 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
Fitting estimator with 8 features.
Fitting estimator with 5 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
Fitting estimator with 8 features.
Fitting estimator with 5 features.
Fitting estimator with 26 features.


array([1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [57]:
np.where(selector.ranking_==1)

(array([ 0,  2,  3,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25]),)

In [63]:
import copy
to_del = []
gold_all_graph=copy.copy(all_graph)
for i,triple in enumerate(training_set):
    source = triple[0]
    target = triple[1]
#     index_source = node_dict[source]
#     index_target = node_dict[target]
#     source_info = node_info[index_source]
#     target_info = node_info[index_target]
#     temp_diff = int(source_info[1]) - int(target_info[1])
#     if temp_diff<-1:
    if triple[2]=='0':
        to_del.append((source,target))

for i,triple in enumerate(testing_set):
    source = triple[0]
    target = triple[1]
    to_del.append((source,target))
        
gold_all_graph.delete_edges(to_del)

In [97]:
import time

t0 = time.time()

small = random.sample(testing_set,5000)
# shortest_paths = [reduced_train_graph.shortest_paths_dijkstra(source = t[0], target = t[1]) for t in small]
sources = list(set([t[0] for t in small]))
targets = list(set([t[1] for t in small]))
shortest_paths_array = np.array( gold_all_graph.shortest_paths_dijkstra(source = sources, 
                                                             target = targets,
                                                             mode = 'OUT') )

# nd_sources = prep.to_dict([s for s in sources],range(len(sources)) )
# nd_targets = prep.to_dict([t for t in targets],range(len(targets)) )

print(0)

shortest_paths_dict = dict(zip( [s for s in sources],
    [dict(zip( [t for t in targets], row)) for row in shortest_paths_array] ))

time.time()-t0

0


9.241418600082397

In [98]:
import time

t0 = time.time()

small2 = random.sample(testing_set,5000)
sources2 = list(set([t[0] for t in small2]))
targets2 = list(set([t[1] for t in small2]))
shortest_paths_array2 = np.array( gold_all_graph.shortest_paths_dijkstra(source = sources2, 
                                                             target = targets2,
                                                             mode = 'OUT') )
print(0)

shortest_paths_dict2 = dict(zip( [s for s in sources2],
    [dict(zip( [t for t in targets2], row)) for row in shortest_paths_array2] ))

time.time()-t0

0


9.25768756866455

In [99]:
nd_sources = prep.to_dict([s for s in sources],range(len(sources)) )
nd_targets = prep.to_dict([t for t in targets],range(len(targets)) )

def path_length(s,t,paths_dict):
    '''
    @paths_dict: shortest paths nested dict where paths_dict[source_id][target_id] is path distance from source to target
    '''
    
    return paths_dict[s][t]

true_pairs = [s for s in small if s[2]=='1']
false_pairs = [s for s in small if s[2]=='0']

In [100]:
feat = [min(path_length(s,t,shortest_paths_dict),10000) for (s,t,b) in small]

feat2 = [min(path_length(s,t,shortest_paths_dict2),10000) for (s,t,b) in small2]

In [69]:
path_dict = prep.all_paths(training_set_reduced,gold_train_graph_reduced)

ValueError: no edge from vertex #18170 to #7629

In [101]:
import other_functions as of
from sklearn.linear_model import LogisticRegression as lr

# of.weighted_overlap(lambda s,t: path_length(s,t,shortest_paths_dict), true_pairs, false_pairs)

model = lr(penalty='l1').fit(np.array(feat).reshape(-1,1), np.array(small)[:,2])
preds_lg = list(model.predict(np.array(feat2).reshape(-1,1)))

# model = lr(penalty='l1').fit(train_sp, train_true_labels)
# preds_lg = list(model.predict(test_sp))

acc = metrics.accuracy_score(list(map(int,np.array(small2)[:,2])), list(map(int,preds_lg)))
# f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_lg)))
print('acc:',acc)


acc: 0.5806


/home/jacob/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [60]:
############################################
###  Construct features on TRAINING_SET  ###
############################################

#Build KDTree on training_set
train_l = [l[node_dict[i]] for i in train_IDs]
train_kdtree = nw.KDTree(train_l)

train_true_labels = []
features_to_create = ['N_LSA_distance']
#'peer_popularity', 'max_sim', 'edge_check', 'title_sim', 'citation_check', 'node_degree','succ_pred', 
    
# Where to insert created features
insert_features_dict = train_features_reduced_dict
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = training_set
title_sim_ones = []
title_sim_zeros = []
for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

    # convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
    # remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]

    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]

    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",") 
    
    # Creating features
    # Baseline #
    #overlap_title = len(set(source_title).intersection(set(target_title)))
    #insert_features_dict["overlap_title"].append(overlap_title)
    #temp_diff = int(source_info[1]) - int(target_info[1])
    #insert_features_dict["temp_diff"].append(temp_diff)
    #comm_auth = len(set(source_auth).intersection(set(target_auth)))
    #insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(reduced_train_graph,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     max_sim = pw.Max_Sim(source,target,l,reduced_train_graph,node_dict)
#     insert_features_dict["max_sim"].append(max_sim)

#     edge_check = pw.edge_check(source,target,reduced_train_graph)
#     insert_features_dict["edge_check"].append(edge_check)

    #LSA_dist = pw.LSA_distance(source,target,node_dict,l)
    #insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     title_weighted = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(title_weighted)
#     if(int(triple[2]) == 0):
#         title_sim_zeros.append(title_weighted)
#     else:
#         title_sim_ones.append(title_weighted)
#     citation_check = pw.Citation_Check(source,target,train_kdtree,l,reduced_train_graph,node_dict,index_dict,k=500)
#     insert_features_dict["citation_check"].append(citation_check)

#     degree = pw.node_degree(source,target,reduced_train_graph)
#     insert_features_dict["node_degree"].append(degree)

#     succ_pred = pw.succ_pred(source,target,reduced_train_graph)
#     insert_features_dict["succ_pred"].append(succ_pred)
    
    N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
    insert_features_dict["N_LSA_distance"].append(N_LSA_dist)
    
#     train_true_labels.append(triple[2])

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
    
for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_train/'+feat)
        
# Concatenate all features
# feats_train = np.concatenate([feat for feat in insert_features_dict.values()])
# train_true_labels = np.array(train_true_labels)

0 / 554602
1000 / 554602
2000 / 554602
3000 / 554602
4000 / 554602
5000 / 554602
6000 / 554602
7000 / 554602
8000 / 554602
9000 / 554602
10000 / 554602
11000 / 554602
12000 / 554602
13000 / 554602
14000 / 554602
15000 / 554602
16000 / 554602
17000 / 554602
18000 / 554602
19000 / 554602
20000 / 554602
21000 / 554602
22000 / 554602
23000 / 554602
24000 / 554602
25000 / 554602
26000 / 554602
27000 / 554602
28000 / 554602
29000 / 554602
30000 / 554602
31000 / 554602
32000 / 554602
33000 / 554602
34000 / 554602
35000 / 554602
36000 / 554602
37000 / 554602
38000 / 554602
39000 / 554602
40000 / 554602
41000 / 554602
42000 / 554602
43000 / 554602
44000 / 554602
45000 / 554602
46000 / 554602
47000 / 554602
48000 / 554602
49000 / 554602
50000 / 554602
51000 / 554602
52000 / 554602
53000 / 554602
54000 / 554602
55000 / 554602
56000 / 554602
57000 / 554602
58000 / 554602
59000 / 554602
60000 / 554602
61000 / 554602
62000 / 554602
63000 / 554602
64000 / 554602
65000 / 554602
66000 / 554602
67000 / 

520000 / 554602
521000 / 554602
522000 / 554602
523000 / 554602
524000 / 554602
525000 / 554602
526000 / 554602
527000 / 554602
528000 / 554602
529000 / 554602
530000 / 554602
531000 / 554602
532000 / 554602
533000 / 554602
534000 / 554602
535000 / 554602
536000 / 554602
537000 / 554602
538000 / 554602
539000 / 554602
540000 / 554602
541000 / 554602
542000 / 554602
543000 / 554602
544000 / 554602
545000 / 554602
546000 / 554602
547000 / 554602
548000 / 554602
549000 / 554602
550000 / 554602
551000 / 554602
552000 / 554602
553000 / 554602
554000 / 554602
overlap_title 27730
comm_auth 27730
temp_diff 27730
citation_check 27730
max_sim 27730
peer_popularity 27730
edge_check 27730
LSA_distance 27730
node_degree 27730
succ_pred 27730
title_sim 27730
temporal_fit 27730
N_LSA_distance 554602


In [144]:
file_path = './features_train/'+'max_sim'+'_reducedgraph'
try:
    this_feat = read_feature(file_path)
except:
    print(1)

In [207]:
to_del = []
reduced_all_graph=copy.copy(all_graph)
for i,triple in enumerate(testing_set):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    source_info = node_info[index_source]
    target_info = node_info[index_target]
    temp_diff = int(source_info[1]) - int(target_info[1])
    if temp_diff<-1:
        to_del.append((source,target))
        
for i,triple in enumerate(training_set):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    source_info = node_info[index_source]
    target_info = node_info[index_target]
    temp_diff = int(source_info[1]) - int(target_info[1])
    if temp_diff<-1:
        to_del.append((source,target))


reduced_all_graph.delete_edges(to_del)

In [57]:
###########################################
###  Construct features on TESTING_SET  ###
###########################################

#Build KDTree on training_set
kdtree = nw.KDTree(l)

train_true_labels = []
features_to_create = ['N_LSA_distance']
#'peer_popularity', 'max_sim', 'edge_check', 'title_sim', 'citation_check','node_degree','succ_pred',

# Where to insert created features
insert_features_dict = test_features_dict
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = testing_set

test_true_labels = []
for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

    # convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
    # remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]

    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]

    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",") 
    
    
    # Creating features
    # Baseline #
    #overlap_title = len(set(source_title).intersection(set(target_title)))
    #insert_features_dict["overlap_title"].append(overlap_title)
    #temp_diff = int(source_info[1]) - int(target_info[1])
    #insert_features_dict["temp_diff"].append(temp_diff)
    #comm_auth = len(set(source_auth).intersection(set(target_auth)))
    #insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(reduced_all_graph,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     max_sim = pw.Max_Sim(source,target,l,reduced_all_graph,node_dict)
#     insert_features_dict["max_sim"].append(max_sim)

#     edge_check = pw.edge_check(source,target,reduced_all_graph)
#     insert_features_dict["edge_check"].append(edge_check)

    #LSA_dist = pw.LSA_distance(source,target,node_dict,l)
    #insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     title_weighted = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(title_weighted)
    
#     citation_check = pw.Citation_Check(source,target,kdtree,l,reduced_all_graph,node_dict,index_dict,k=500)
#     insert_features_dict["citation_check"].append(citation_check)
    
#     succ_pred = pw.succ_pred(source,target,reduced_all_graph)
#     insert_features_dict["succ_pred"].append(succ_pred)
    
#     degree = pw.node_degree(source,target,reduced_all_graph)
#     insert_features_dict["node_degree"].append(degree)
    
    N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
    insert_features_dict["N_LSA_distance"].append(N_LSA_dist)

#     test_true_labels.append(triple[2])

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
        
# Concatenate all features
# feats_test = np.concatenate([feat for feat in insert_features_dict.values()])
# test_true_labels = np.array(test_true_labels)

for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_test/'+feat)

0 / 60910
1000 / 60910
2000 / 60910
3000 / 60910
4000 / 60910
5000 / 60910
6000 / 60910
7000 / 60910
8000 / 60910
9000 / 60910
10000 / 60910
11000 / 60910
12000 / 60910
13000 / 60910
14000 / 60910
15000 / 60910
16000 / 60910
17000 / 60910
18000 / 60910
19000 / 60910
20000 / 60910
21000 / 60910
22000 / 60910
23000 / 60910
24000 / 60910
25000 / 60910
26000 / 60910
27000 / 60910
28000 / 60910
29000 / 60910
30000 / 60910
31000 / 60910
32000 / 60910
33000 / 60910
34000 / 60910
35000 / 60910
36000 / 60910
37000 / 60910
38000 / 60910
39000 / 60910
40000 / 60910
41000 / 60910
42000 / 60910
43000 / 60910
44000 / 60910
45000 / 60910
46000 / 60910
47000 / 60910
48000 / 60910
49000 / 60910
50000 / 60910
51000 / 60910
52000 / 60910
53000 / 60910
54000 / 60910
55000 / 60910
56000 / 60910
57000 / 60910
58000 / 60910
59000 / 60910
60000 / 60910
overlap_title 60910
comm_auth 60910
temp_diff 60910
citation_check 60910
max_sim 60910
peer_popularity 60910
edge_check 60910
LSA_distance 60910
node_degree 60

NameError: name 'write_feature_to_disk' is not defined

In [61]:
###########################################
###  Construct features on COMPETITION_SET  ###
###########################################

#Build KDTree on training_set
kdtree = nw.KDTree(l)

features_to_create = ["N_LSA_distance"]
# "overlap_title",
#                  "comm_auth",
#                  "temp_diff",
#                 "citation_check",
#                 "max_sim",
#                 "peer_popularity",
#                 "edge_check",
#                 "LSA_distance",
#                 "node_degree",
#                 "succ_pred",
#                 "title_sim",
#                 "temporal_fit",

# Where to insert created features
insert_features_dict = dict()
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = competition_set

for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

    # convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
    # remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]

    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]

    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",") 
    
    
    # Creating features
    # Baseline #
#     overlap_title = len(set(source_title).intersection(set(target_title)))
#     insert_features_dict["overlap_title"].append(overlap_title)
#     temp_diff = int(source_info[1]) - int(target_info[1])
#     insert_features_dict["temp_diff"].append(temp_diff)
#     comm_auth = len(set(source_auth).intersection(set(target_auth)))
#     insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(competition_graph_reduced,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     max_sim = pw.Max_Sim(source,target,l,competition_graph_reduced,node_dict)
#     insert_features_dict["max_sim"].append(max_sim)

#     edge_check = pw.edge_check(source,target,competition_graph_reduced)
#     insert_features_dict["edge_check"].append(edge_check)

#     LSA_dist = pw.LSA_distance(source,target,node_dict,l)
#     insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     citation_check = pw.Citation_Check(source,target,kdtree,l,competition_graph_reduced,node_dict,index_dict,k=500)
#     insert_features_dict["citation_check"].append(citation_check)
    
#     succ_pred = pw.succ_pred(source,target,competition_graph_reduced)
#     insert_features_dict["succ_pred"].append(succ_pred)
    
#     degree = pw.node_degree(source,target,competition_graph_reduced)
#     insert_features_dict["node_degree"].append(degree)
    
#     ts = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(ts)
    
#     temporal_fit = pw.temp_fit(source,target,competition_graph_reduced,node_dict,publication_years)
#     insert_features_dict["temporal_fit"].append(temporal_fit)
    
    N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
    insert_features_dict["N_LSA_distance"].append(N_LSA_dist)

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
        
# Concatenate all features
# feats_test = np.concatenate([feat for feat in insert_features_dict.values()])
# test_true_labels = np.array(test_true_labels)

for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_competition/'+feat+'_reducedgraph')

0 / 32648
1000 / 32648
2000 / 32648
3000 / 32648
4000 / 32648
5000 / 32648
6000 / 32648
7000 / 32648
8000 / 32648
9000 / 32648
10000 / 32648
11000 / 32648
12000 / 32648
13000 / 32648
14000 / 32648
15000 / 32648
16000 / 32648
17000 / 32648
18000 / 32648
19000 / 32648
20000 / 32648
21000 / 32648
22000 / 32648
23000 / 32648
24000 / 32648
25000 / 32648
26000 / 32648
27000 / 32648
28000 / 32648
29000 / 32648
30000 / 32648
31000 / 32648
32000 / 32648
N_LSA_distance 32648
